### Array Operations

#### `map` and `broadcast` are great!

To apply the anonymous function (x)->x^2 to each element,

In [ ]:
map((x)->x^2,1:5)

If we need to add a vector to each column of a matrix, we can use `broadcast`. This is useful to perform element-wise operations on arrays of different sizes.

In [ ]:
A = 1:5 # Acts like a column vector, Julia is "column-major" so columns come first
B = [1 2
     3 4
     5 6
     7 8
     9 10]
broadcast(+,A,B)

In [ ]:
A = 1:5
B = [2;3;4;5;6]
A.*B

In [ ]:
C = [3;4;5;2;1]
A.*B.*C

In [ ]:
broadcast((x,y,z)->x*y*z,A,B,C)

because all array-based math uses this broadcasting syntax with a ., Julia can fuse the broadcasts on all sorts of mathematical expressions on arrays

In [ ]:
A.*B.*sin.(C)

One last thing to note is that we can also broadcast =. This would be the same thing is as the loop A[i] = ... and thus requires the array A to already be define. Thus for example, if we let

In [ ]:
D = similar(C)

In [ ]:
@time D.=A.*B.*C

The above operation does not allocate any arrays. Reducing temporary array allocations is one way Julia outperforms other scientific computing languages.

### Vectors and Matrices 

In [ ]:
A = rand(4,4) # Generate a 4x4 random matrix
A[1:3,1:3] # Take the top left 3-3 matrix

Note that Julia is column-major, meaning that columns come first in both indexing order and in the computer's internal representation.

### Views

Notice that A[1:3,1:3] returned an array. Where did this array come from? Well, since there was no 3x3 array before, A[1:3,1:3] created an array (i.e. it had to allocate memory)

In [ ]:
@time A[1:3,1:3]

Allocation of memory while creating variables,

In [ ]:
a = [1;3;5]
@time b = a
a[2] = 10
a
@time c = copy(a)

In the first case it just created a pointer to object of `a`. In Julia an array is actually an array in the memory layout, which is actually a C-pointer to a contiguous 1-dimensional slots of memory. For example,

In [ ]:
A = rand(4,4)

This is a 16 number of consecutive memory slots. and `A` is a view to that, indexed in sucha way to make it look like a 2-dimensional array.

In [ ]:
function testloops()
    b = rand(1000,1000)
    c = 0 # Need this so that way the compiler doesn't optimize away the loop!
    @time for i in 1:1000, j in 1:1000
        c+=b[i,j]
    end
    @time for j in 1:1000, i in 1:1000
        c+=b[i,j]
    end
    bidx = eachindex(b)
    @time for i in bidx
        c+=b[i]
    end
end
testloops()

One should normally use the eachindex function since this will return the indices in the "fast" order for general iterator types.

In this terminology A[1:3,1:3] isn't a view to the same memory. We can check this by noticing that it doesn't mutate the original array:

In [ ]:
println(A)
B = A[1:3,1:3]
B[1,1]=100
println(A)

If we instead want a view, then we can use the view function:

In [ ]:
B = view(A,1:3,1:3) # No copy involved
B[1,1] = 100 # Will mutate A
println(A)

There are many cases where you might want to use a view. For example, if a function needs the ith column, you may naively think of doing f(A[i,:]). But, if A won't be changed in the loop, we can avoid the memory allocation (and thus make things faster) by sending a view to the original array which is simply the column: f(view(A,i,:)). Two functions can be used to give common views. vec gives a view of the array as a Vector and reshape builds a view in a different shape. For example:

In [ ]:
C = vec(A)
println(C)
C = reshape(A,8,2) # C is an 8x2 matrix
C